In [233]:
import numpy as np
import re

# Documentation for various intcodes
def documentation():
    
    print("A user input is needed to run this Intcode.")
    print("Please run as: 'IntcodeComputer(Intcode, USER_INPUT)'")
    print("where USER_INPUT is an integer. Use:")
    print(" - 1 for diagnostics")
    print(" - 5 for Thermal Radiator Controller ")
    
    return 0


# A structured data type for instructions
class Instruction:
        
    def __init__(self, code):
        
        # Sets that associates an opcode with its associated number of parameters
        # 3 parameters opcodes
        III_params = [1, 2, 7, 8]
        # 2 parameters opcodes
        II_params = [5, 6]
        # 1 parameter opcodes
        I_params = [3, 4]
        
        # Variables of the instruction set
        # Increment value after instruction
        self.increment = 0
        # Number of parameters
        self.n_params = 0
        # Parameter modes
        self.param_mode = []
        # Control variable
        self.is_corrupted = False
        # Number of digits that make up the opcode
        opcode_digits = 2
        
        lscode = list(str(code))
        self.opcode = int(''.join(lscode[-opcode_digits:])) # This works even if len(o) = 1!
        
        if self.opcode in III_params:
            self.increment = 4
            self.n_params = 3
            # My favorite line <3
            #self.param_mode = [int(x) if len(o) > 2 else 0 for x in reversed(o[:-2])]
        elif self.opcode in I_params:
            self.increment = 2
            self.n_params = 1
            
        elif self.opcode in II_params:
            self.increment = 3
            self.n_params = 2
       
        elif self.opcode == 99:
            pass
            
        else:
            self.is_corrupted = True
                    
        for i in range(self.n_params):
            try:
                rlscode = lscode[::-1]    # Reversed 'lscode'
                self.param_mode.append(int(rlscode[opcode_digits + i]))
                
            except:
                self.param_mode.append(0)
         
    def ShowInstructionData(self, code):
        print("Instruction: ", code)
        print("Opcode: ", self.opcode)
        print("Increment: ", self.increment)
        print("Number of parameters: ", self.n_params)
        print("parameters mode: ", self.param_mode)
        
                
# The Intcode computer
def IntcodeComputer(sequence, user_input = 'None'):
    
    Intcode_length = len(sequence)
    curr_pos = 0
    
    i = Instruction(sequence[curr_pos])
    while(i.opcode != 99): 
        
        #i.ShowInstructionData(sequence[curr_pos])
        if not i.is_corrupted:
            
            # Initialize variables if needed for execution of the instruction
            if i.n_params > 0:
                a = sequence[curr_pos+1] if i.param_mode[0] else sequence[sequence[curr_pos+1]]
            if i.n_params >= 2:
                b = sequence[curr_pos+2] if i.param_mode[1] else sequence[sequence[curr_pos+2]]
   
            # Add
            if i.opcode == 1:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = a+b
                else:
                    sequence[sequence[curr_pos+3]] = a+b
                    
                curr_pos += i.increment
            
            # Multiply
            elif i.opcode == 2:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = a*b
                else:
                    sequence[sequence[curr_pos+3]] = a*b
                    
                curr_pos += i.increment
                
            # Write input to address    
            elif i.opcode == 3:
                if user_input != 'None':
                    sequence[sequence[curr_pos+1]] = user_input
                else:
                    documentation()
                    return 0
                    
                curr_pos += i.increment
                
            
            # Print output from address
            elif i.opcode == 4:
                output = sequence[curr_pos+1] if i.param_mode[0] else sequence[sequence[curr_pos+1]]
                
                #if output != 0:
                #    print('*******  debugger  ********')
                #    i.ShowInstructionData(sequence[curr_pos])
                    
                print("Output: ", output, '\n')
                
                curr_pos += i.increment
             
            # jump-if-True
            elif i.opcode == 5:
                if a != 0:
                    curr_pos = b
                else:
                    curr_pos += i.increment
            
            # jump-if-false
            elif i.opcode == 6:
                if a == 0:
                    curr_pos = b
                else:
                    curr_pos += i.increment
            
            # less-than
            elif i.opcode == 7:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = 1 if a < b else 0
                else:
                    sequence[sequence[curr_pos+3]] = 1 if a < b else 0
                    
                curr_pos += i.increment
            
            # equal
            elif i.opcode == 8:
                if i.param_mode[2]:
                    sequence[curr_pos+3] = 1 if a == b else 0
                else:
                    sequence[sequence[curr_pos+3]] = 1 if a == b else 0
                    
                curr_pos += i.increment
        
        # If opcode is unknown
        else:
            print("Unknown opcode '" + str(i.opcode) + "'. Terminating.")
            return -1
   
        # Read the future instruction
        i = Instruction(sequence[curr_pos])
        #print(sequence)
     
    print("Program is finished. Haulting.")
    
    return 0


In [234]:
with open("input.txt", 'r') as infile:
    strIntcode = infile.read().split(',')
    
# Load memory from file
Intcode = np.array([int(item) for item in strIntcode])
#print(Intcode)
# Modify input memory according to assignement
#Intcode[1] = 12
#Intcode[2] = 2

# Run instructions, and save in variable success
success = IntcodeComputer(Intcode, 5)

Output:  5000972 

Program is finished. Haulting.


In [227]:
a = [3,3,1105,-1,9,1101,0,0,12,4,12,99,1]
3,3,1107,-1,8,3,4,3,99
IntcodeComputer(a, 0)

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1101, 0, 0, 12, 4, 12, 99, 1]
[3, 3, 1105, 0, 9, 1

KeyboardInterrupt: 

In [182]:
if 8:
    print('H')

H
